Importandos todas as Bibliotecas necessárias.

In [53]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx2pdf import convert
import time

Classe que realiza a busca de dados na planilha do Excel e retorna um dicionário:<br>
 * Se não for especificada a aba, ele irá buscar em todas as abas até encontrar a primeira ocorrência;

 * Se a célula for vazia, a mesma será excluído do dicionário;
 
 * Importante analisar a ordem das abas, abas com valores únicos devem ser as primeiras da planilha;

 * Se na célular houver valores separados por ';' será criada uma lista com os valores separados pelo ';'.

In [4]:
class BuscarDados:

    def __init__(self, valor, aba=None):
        
        if not aba:
            banco_de_dados = pd.read_excel('01_Cadastro.xlsm', sheet_name=None)
            for dados_aba in banco_de_dados.values():
                if valor in dados_aba.values:
                    banco_de_dados = dados_aba
                    break
            else:
                raise ValueError('O valor não foi encontrado nas abas do arquivo')

        else:
            banco_de_dados = pd.read_excel('01_Cadastro.xlsm', sheet_name = aba)
            if valor not in banco_de_dados.values:
                raise ValueError('O valor não foi encontrado nas abas do arquivo')

        linha = banco_de_dados.loc[banco_de_dados.isin([valor]).any(axis = 1)].iloc[0].dropna().astype(str)
        self.dicionario = {indice: valor.split(';') if ';' in valor
                            else valor for indice, valor in linha.items()}
        
        if 'ATRIBUTOS' in self.dicionario:            
            if isinstance(self.dicionario['ATRIBUTOS'], str):
                self.dicionario[self.dicionario.pop('ATRIBUTOS')] = self.dicionario.pop('VALORES')
            else:
                self.dicionario.update(dict(zip(self.dicionario.pop('ATRIBUTOS'), self.dicionario.pop('VALORES'))))

Classe que cria um dicionária da Parte procurada:
* As chaves do dicionário são os atributos da parte, com seus valores como os valores;

* Qualquer atributo pode ser usado para buscar a parte (nome, cpf, cnpj, etc...).

In [5]:
class QualificarParte:
    
    def __init__(self, parte):
        
        self.dicionario = BuscarDados(parte).dicionario
        self._agrupar_endereco()
        self._qualificar_parte()
          
    def _agrupar_endereco(self):
        
        self.dicionario['NUMERO_COMPLEMENTO'] = (f"{self.dicionario['NUMERO']} - {self.dicionario['COMPLEMENTO']}"
                                                if 'COMPLEMENTO' in self.dicionario else self.dicionario['NUMERO'])
                
        self.dicionario['ENDERECO'] = '{}, nº {}, Bairro {}, CEP {}, {}/{}'.format(
                                        self.dicionario['LOGRADOURO'], self.dicionario['NUMERO_COMPLEMENTO'],
                                        self.dicionario['BAIRRO'], self.dicionario['CEP'],
                                        self.dicionario['CIDADE'], self.dicionario['UF']
                                        )
   
    def _qualificar_parte(self):

        if 'CPF' in self.dicionario:
            self.dicionario['QUALIFICAÇÃO'] = '{}, {}, {}, {}, RG n° {} {}, CPF n° {}, e-mail: {}, com endereço à {}'.format(
                                                self.dicionario['NOME'], self.dicionario['NACIONALIDADE'], self.dicionario['ESTADO_CIVIL'],
                                                self.dicionario['PROFISSAO'], self.dicionario['RG'], self.dicionario['EXPEDIDOR_RG'],
                                                self.dicionario['CPF'], self.dicionario['EMAIL'],self.dicionario['ENDERECO']
                                                )

        elif 'TAG' in self.dicionario and self.dicionario['TAG'] == 'condominio':
            self.dicionario['QUALIFICAÇÃO'] = ('{}, inscrito no CNPJ n° {}, com endereço à {}, devidamente representado neste ato '
                                               'pelo seu síndico(a) {}'.format(
                                                self.dicionario['NOME'], self.dicionario['CNPJ'],
                                                self.dicionario['ENDERECO'], self.dicionario['ADMINISTRADOR'])
                                                )
            
        else:
            self.dicionario['QUALIFICAÇÃO'] = ('{}, pessoa jurídica inscrita no CNPJ n° {}, com endereço à {}, devidamente '
                                                'representada neste ato pelo(a) sócio(a) {}').format(
                                                self.dicionario['NOME'], self.dicionario['CNPJ'],
                                                self.dicionario['ENDERECO'], self.dicionario['ADMINISTRADOR']
                                                )

Classe que cria um dicionário do documento procurado:
* O documento deve ser procurado pelo ID (pois é o atributo único dele);

* O cliente ou o adverso podem ser mais de uma pessoa;

* O adverso é opcional.

In [6]:
class CriarDocumento:
    
    def __init__(self, id_documento):
        
        self.dicionario = BuscarDados(id_documento, 'Peticoes').dicionario
        self._dicionario_parte(self.dicionario['CLIENTE'])
        if 'ADVERSO' in self.dicionario:
            self._dicionario_parte(self.dicionario['ADVERSO'], 'ADVERSO_')

    def _dicionario_parte(self, parte_do_documento, prefixo = 'CLIENTE_'):
        
        def dicionario_se_parte_unica(parte_do_documento, prefixo):
            for chave, valor in QualificarParte(parte_do_documento).dicionario.items():
                nova_chave = prefixo + chave
                self.dicionario[nova_chave] = valor
        
        # Checa o número de parte se é multiplo (lista).
        if isinstance(parte_do_documento, str):
            dicionario_se_parte_unica(parte_do_documento, prefixo)
        
        else:
            dicionario_se_parte_unica(parte_do_documento[0], prefixo)
            self.dicionario[prefixo+'QUALIFICAÇÃO'] = f"(1){self.dicionario[prefixo+'QUALIFICAÇÃO']}"
            self.dicionario['(1)'+prefixo+'QUALIFICAÇÃO'] = self.dicionario[prefixo+'QUALIFICAÇÃO']
            
            for index, nome in enumerate(parte_do_documento[1:], start=2):
                self.dicionario[prefixo+'QUALIFICAÇÃO'] += f" e ({index}){QualificarParte(nome).dicionario['QUALIFICAÇÃO']}"
                prefixo2 = f'({index})' + prefixo
                dicionario_se_parte_unica(nome, prefixo2)

Substitui os placeholders(chaves) de um arquivo docx pelos valores contidos nas chaves do dicionário criado com 'CriarDocumento'.<br>

Na sequência, salva em um arquivo docx em uma pasta já especificada e converte também em pdf.

In [72]:
class CriarDocxPdf:

    def __init__(self, id_documento):
        
        dicionario = CriarDocumento(id_documento).dicionario
        docx = Document(f"01_Modelos\{dicionario['PECA']}.docx")
        dicionario_de_substituicao = {'x' + chave.upper() + 'x': valor for chave, valor in dicionario.items()}
        
        for paragrafo in docx.paragraphs:
            for chave in dicionario_de_substituicao:
                if chave in paragrafo.text:
                    paragrafo.text = paragrafo.text.replace(chave, dicionario_de_substituicao[chave])
        
        nome_salvo = (f"{dicionario['PECA']} - {dicionario['CLIENTE_NOME']} x {dicionario['ADVERSO_NOME']}"
                    if 'ADVERSO_NOME' in dicionario else f"{dicionario['PECA']} - {dicionario['CLIENTE_NOME']}")

        docx.save(f"02_Documentos-prontos\{nome_salvo}.docx")
        convert(f"02_Documentos-prontos\{nome_salvo}.docx", f"02_Documentos-prontos\{nome_salvo}.pdf")

In [ ]:
CriarDocxPdf(1)
CriarDocxPdf(2)
CriarDocxPdf(3)

In [10]:
class processo:

    def __init__(self, processo):

        self.dicionario = busca_dados(processo).linha.to_dict()

class peca:

    def __init__(self, peca):

        self.dicionario = busca_dados(peca).linha.to_dict()

In [4]:
from num2words import num2words

numero = 23
extenso = num2words(numero, lang='pt_BR')

print(f'{extenso} reais e {extenso} centavos')


vinte e três reais e vinte e três centavos


In [11]:
import pyodbc

dados_conexao = ('Driver={SQL Server};Server=DESKTOP-SMFGJPJ\SQLEXPRESS;Database=teste')
conexao = pyodbc.connect(dados_conexao)
print('Conexão Bem sucedida')
cursor = conexao.cursor()
cursor.execute("SELECT * FROM nome")
cursor.commit()
cursor.close()
conexao.close()


Conexão Bem sucedida


In [ ]:
cursor.execute("INSERT INTO tabela_origem (campo1, campo2) VALUES (?, ?)", valor1, valor2)

# Obter o ID do último registro inserido
cursor.execute("SELECT SCOPE_IDENTITY() AS LastID")
id_origem = cursor.fetchone().LastID

# Inserir dados na tabela de destino usando o ID da tabela de origem
cursor.execute("INSERT INTO tabela_destino (id_origem, campo3, campo4) VALUES (?, ?, ?)", id_origem, valor3, valor4)

# Confirmar as alterações
cursor.commit()


